# Assignment 3

Welcome to Assignment 3. This will be even more fun. Now we will calculate statistical measures. 

## You only have to pass 4 out of 7 functions

Just make sure you hit the play button on each cell from top to down. There are seven functions you have to implement. Please also make sure than on each change on a function you hit the play button again on the corresponding cell to make it available to the rest of this notebook.

This notebook is designed to run in a IBM Watson Studio default runtime (NOT the Watson Studio Apache Spark Runtime as the default runtime with 1 vCPU is free of charge). Therefore, we install Apache Spark in local mode for test purposes only. Please don't use it in production.

In case you are facing issues, please read the following two documents first:

https://github.com/IBM/skillsnetwork/wiki/Environment-Setup

https://github.com/IBM/skillsnetwork/wiki/FAQ

Then, please feel free to ask:

https://coursera.org/learn/machine-learning-big-data-apache-spark/discussions/all

Please make sure to follow the guidelines before asking a question:

https://github.com/IBM/skillsnetwork/wiki/FAQ#im-feeling-lost-and-confused-please-help-me


If running outside Watson Studio, this should work as well. In case you are running in an Apache Spark context outside Watson Studio, please remove the Apache Spark setup in the first notebook cells.

In [1]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown('# <span style="color:red">'+string+'</span>'))


if ('sc' in locals() or 'sc' in globals()):
    printmd('<<<<<!!!!! It seems that you are running in a IBM Watson Studio Apache Spark Notebook. Please run it in an IBM Watson Studio Default Runtime (without Apache Spark) !!!!!>>>>>')


In [2]:
!pip install pyspark==3.3.0

     |████████████████████████████████| 281.3 MB 96 kB/s s eta 0:00:01        | 46.8 MB 56.6 MB/s eta 0:00:05                | 65.2 MB 56.6 MB/s eta 0:00:04�█████████▏                 | 125.0 MB 65.4 MB/s eta 0:00:03��██████████████              | 158.3 MB 41.2 MB/s eta 0:00:03:00:02
     |████████████████████████████████| 199 kB 60.5 MB/s eta 0:00:01
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764027 sha256=4fcc9cbb15cf517a26868199b42e99de713c00d62be0fa792d9ee44769401cb1
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/05/75/73/81f84d174299abca38dd6a06a5b98b08ae25fce50ab8986fa1
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.2
    Uninstalling py4j-0.10.9.2:
      Successfully uninstalled py4j-0.10.9.2


In [3]:
try:
    from pyspark import SparkContext, SparkConf
    from pyspark.sql import SparkSession
except ImportError as e:
    printmd('<<<<<!!!!! Please restart your kernel after installing Apache Spark !!!!!>>>>>')

In [5]:
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

spark = SparkSession \
    .builder \
    .getOrCreate()

All functions can be implemented using DataFrames, ApacheSparkSQL or RDDs. We are only interested in the result. You are given the reference to the data frame in the "df" parameter and in case you want to use SQL just use the "spark" parameter which is a reference to the global SparkSession object. Finally if you want to use RDDs just use "df.rdd" for obtaining a reference to the underlying RDD object. But we discurage using RDD at this point in time.

Let's start with the first function. Please calculate the minimal temperature for the test data set you have created. We've provided a little skeleton for you in case you want to use SQL. Everything can be implemented using SQL only if you like.

In [12]:
def minTemperature(df,spark):
 minRow=df.agg({"temperature": "min"}).collect()[0]
 mintemp = minRow["min(temperature)"]
#    print mintemp
 return mintemp

Please now do the same for the mean of the temperature

In [13]:
def meanTemperature(df,spark):
    avgRow=df.agg({"temperature": "avg"}).collect()[0]
    avgtemp = avgRow["avg(temperature)"]
#    print avgtemp
    return avgtemp

Please now do the same for the maximum of the temperature

In [14]:
def maxTemperature(df,spark):
    maxRow=df.agg({"temperature": "max"}).collect()[0]
    maxtemp = maxRow["max(temperature)"]
#    print maxtemp
    return maxtemp

Please now do the same for the standard deviation of the temperature

In [18]:
def sdTemperature(df,spark):
    temprddrow = df.select('temperature').rdd #in row(temp=x) format
    temprdd = temprddrow.map(lambda x : x["temperature"]) #only numbers
    temp = temprdd.filter(lambda x: x is not None).filter(lambda x: x != "") #remove None params
    n = float(temp.count())
    sum=temp.sum()
    mean =sum/n
    from math import sqrt
    sd=sqrt(temp.map(lambda x : pow(x-mean,2)).sum()/n)
#    print sd
    return sd

Please now do the same for the skew of the temperature. Since the SQL statement for this is a bit more complicated we've provided a skeleton for you. You have to insert custom code at four positions in order to make the function work. Alternatively you can also remove everything and implement if on your own. Note that we are making use of two previously defined functions, so please make sure they are correct. Also note that we are making use of python's string formatting capabilitis where the results of the two function calls to "meanTemperature" and "sdTemperature" are inserted at the "%s" symbols in the SQL string.

In [20]:
def skewTemperature(df,spark):    
    temprddrow = df.select('temperature').rdd #in row(temp=x) format
    temprdd = temprddrow.map(lambda x : x["temperature"]) #only numbers
    temp = temprdd.filter(lambda x: x is not None).filter(lambda x: x != "")  #remove None params
    n = float(temp.count())
    sum=temp.sum()
    mean =sum/n
    from math import sqrt
    sd=sqrt(temp.map(lambda x : pow(x-mean,2)).sum()/n)
    skew=n*(temp.map(lambda x:pow(x-mean,3)/pow(sd,3)).sum())/(float(n-1)*float(n-2))
#    print skew
    return skew 

Kurtosis is the 4th statistical moment, so if you are smart you can make use of the code for skew which is the 3rd statistical moment. Actually only two things are different.

In [21]:

def kurtosisTemperature(df,spark): 
    temprddrow = df.select('temperature').rdd
    temprdd = temprddrow.map(lambda x : x["temperature"])
    temp = temprdd.filter(lambda x: x is not None).filter(lambda x: x != "")
    n = float(temp.count())
    sum=temp.sum()
    mean =sum/n
    from math import sqrt
    sd=sqrt(temp.map(lambda x : pow(x-mean,2)).sum()/n)
    kurtosis=temp.map(lambda x:pow(x-mean,4)).sum()/(pow(sd,4)*(n))
#    print kurtosis
    return kurtosis


Just a hint. This can be solved easily using SQL as well, but as shown in the lecture also using RDDs.

In [25]:
def correlationTemperatureHardness(df,spark):
    temprddrow = df.select('temperature').rdd
    temprdd = temprddrow.map(lambda x : x["temperature"])
    temp = temprdd.filter(lambda x: x is not None).filter(lambda x: x != "")
    hardrddrow = df.select('hardness').rdd
    hardrdd = hardrddrow.map(lambda x : x["hardness"])
    hard = hardrdd.filter(lambda x: x is not None).filter(lambda x: x != "")
    n = float(temp.count())
    sumt=temp.sum()
    sumh=hard.sum()
    meant =sumt/n
    meanh = sumh/n
    rdd12 = temp.zip(hard)
    cov12 = rdd12.map(lambda x,y : (x-meant)*(y-meanh)).sum()/n
    from math import sqrt
    sd1 = sqrt(temp.map(lambda x : pow(x-meant,2)).sum()/n)
    sd2 = sqrt(hard.map(lambda x : pow(x-meanh,2)).sum()/n)
    corr12 = cov12 / (sd1 * sd2)
    #corr12
    return corr12

Now it is time to grab a PARQUET file and create a dataframe out of it. Using SparkSQL you can handle it like a database. 

In [26]:
!wget https://github.com/IBM/coursera/blob/master/coursera_ds/washing.parquet?raw=true
!mv washing.parquet?raw=true washing.parquet

--2022-10-12 14:52:11--  https://github.com/IBM/coursera/blob/master/coursera_ds/washing.parquet?raw=true
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/IBM/claimed/blob/master/coursera_ds/washing.parquet?raw=true [following]
--2022-10-12 14:52:11--  https://github.com/IBM/claimed/blob/master/coursera_ds/washing.parquet?raw=true
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/IBM/claimed/raw/master/coursera_ds/washing.parquet [following]
--2022-10-12 14:52:11--  https://github.com/IBM/claimed/raw/master/coursera_ds/washing.parquet
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/IBM/claimed/master/coursera_ds/washing.parquet [following]
--2022-10-12 14:52:

In [28]:
df = spark.read.parquet('washing.parquet')
df.createOrReplaceTempView('washing')
df.show()

+--------------------+--------------------+-----+--------+----------+---------+--------+-----+-----------+-------------+-------+
|                 _id|                _rev|count|flowrate|fluidlevel|frequency|hardness|speed|temperature|           ts|voltage|
+--------------------+--------------------+-----+--------+----------+---------+--------+-----+-----------+-------------+-------+
|0d86485d0f88d1f9d...|1-57940679fb8a713...|    4|      11|acceptable|     null|      77| null|        100|1547808723923|   null|
|0d86485d0f88d1f9d...|1-15ff3a0b304d789...|    2|    null|      null|     null|    null| 1046|       null|1547808729917|   null|
|0d86485d0f88d1f9d...|1-97c2742b68c7b07...|    4|    null|      null|       71|    null| null|       null|1547808731918|    236|
|0d86485d0f88d1f9d...|1-eefb903dbe45746...|   19|      11|acceptable|     null|      75| null|         86|1547808738999|   null|
|0d86485d0f88d1f9d...|1-5f68b4c72813c25...|    7|    null|      null|       75|    null| null|   

Now let's test the functions you've implemented

In [31]:
min_temperature = 0
mean_temperature = 0
max_temperature = 0
sd_temperature = 0
skew_temperature = 0
kurtosis_temperature = 0
correlation_temperature = 0


In [33]:
min_temperature = minTemperature(df,spark)
print(min_temperature)

80


In [35]:
mean_temperature = meanTemperature(df,spark)
print(mean_temperature)

90.03800298062593


In [36]:
max_temperature = maxTemperature(df,spark)
print(max_temperature)

100


In [37]:
sd_temperature = sdTemperature(df,spark)
print(sd_temperature)

6.098487624200329


In [38]:
skew_temperature = skewTemperature(df,spark)
print(skew_temperature)

0.010433319824286571


In [39]:
kurtosis_temperature = kurtosisTemperature(df,spark)
print(kurtosis_temperature)

1.7760730694213247


In [52]:
correlation_temperature = correlationTemperatureHardness(df, spark)
print(correlation_temperature)

22/10/12 14:59:15 ERROR Executor: Exception in task 0.0 in stage 39.0 (TID 36)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 686, in main
    process()
  File "/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 676, in process
    out_iter = func(split_index, iterator)
  File "/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/pyspark/rdd.py", line 3472, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/pyspark/rdd.py", line 3472, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/pyspark/rdd.py", line 540, in func
    return f(iterator)
  File "/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/pyspark/rdd.py"

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 39.0 failed 1 times, most recent failure: Lost task 0.0 in stage 39.0 (TID 36) (notebook-conda1rt221acf2aea37a554f09a691c9b0472a9a7f-8c5f7qzwf5 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 686, in main
    process()
  File "/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 676, in process
    out_iter = func(split_index, iterator)
  File "/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/pyspark/rdd.py", line 3472, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/pyspark/rdd.py", line 3

Congratulations, you are done, please submit this notebook to the grader. 
We have to install a little library in order to submit to coursera first.

Then, please provide your email address and obtain a submission token on the grader’s submission page in coursera, then execute the subsequent cells

### Note: We've changed the grader in this assignment and will do so for the others soon since it gives less errors
This means you can directly submit your solutions from this notebook

In [46]:
!rm -f rklib.py
!wget https://raw.githubusercontent.com/IBM/coursera/master/rklib.py

--2022-10-12 14:57:12--  https://raw.githubusercontent.com/IBM/coursera/master/rklib.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2540 (2.5K) [text/plain]
Saving to: ‘rklib.py’

rklib.py            100%[===================>]   2.48K  --.-KB/s    in 0s      

2022-10-12 14:57:12 (35.7 MB/s) - ‘rklib.py’ saved [2540/2540]



In [49]:
from rklib import submitAll
import json

key = "Suy4biHNEeimFQ479R3GjA"
email = "atharvabargir@gmail.com"
token = "IzqE84s1mvuH8DkW"###_YOUR_CODE_GOES_HERE_### #you can obtain it from the grader page on Coursera




In [50]:
parts_data = {}
parts_data["FWMEL"] = json.dumps(min_temperature)
parts_data["3n3TK"] = json.dumps(mean_temperature)
parts_data["KD3By"] = json.dumps(max_temperature)
parts_data["06Zie"] = json.dumps(sd_temperature)
parts_data["Qc8bI"] = json.dumps(skew_temperature)
parts_data["LoqQi"] = json.dumps(kurtosis_temperature)
parts_data["ehNGV"] = json.dumps(correlation_temperature)



submitAll(email, token, key, parts_data)

Submission successful, please check on the coursera grader page for the status
-------------------------
{"elements":[{"itemId":"TzU1P","id":"sUpST4RAEeawAApvKZgcCQ~TzU1P~YHfgHko-Ee211hKE27R-sQ","courseId":"sUpST4RAEeawAApvKZgcCQ"}],"paging":{},"linked":{}}
-------------------------
